In [1]:
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import norm, mannwhitneyu

In [2]:
exp_root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged"

df_results = pd.read_csv(os.path.join(exp_root, "results.csv"))

df_results

,model,task,dataset,model_type,trainable_module,path,have_eval_result,have_prediction,have_gpt_score,model_name,...,exact_match_overall,recall_overall,precision_overall,f1_overall,exact_match_closed,recall_closed,precision_closed,f1_score_closed,accuracy_closed,gpt_score_open
0,Qwen2-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen2-VL/eval_seed0/Qwen2-VL-7B-Inst...,1,1,0,Qwen2-VL-7B-Instruct,...,0.285276,0.624306,0.292802,0.318349,0.658153,0.774038,0.661506,0.668063,0.778846,NaN
1,Qwen25-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen25-VL/eval_seed0/Qwen2.5-VL-7B-I...,1,1,0,Qwen2.5-VL-7B-Instruct,...,0.252990,0.596551,0.260487,0.274167,0.627367,0.747596,0.629623,0.633801,0.754808,NaN
2,Gemma3,vqa,SLAKE,general,NaN,vqa/SLAKE/Gemma3/eval_seed0/gemma-3-4b-it,1,1,0,gemma-3-4b-it,...,0.032464,0.525746,0.038177,0.056352,0.070822,0.657252,0.073086,0.097527,0.721154,NaN
3,MedGemma,vqa,SLAKE,medical,NaN,vqa/SLAKE/MedGemma/eval_seed0/medgemma-4b-it,1,1,0,medgemma-4b-it,...,0.243252,0.575250,0.250568,0.265892,0.597256,0.725962,0.599219,0.604106,0.769231,NaN
4,InternVL3,vqa,SLAKE,general,NaN,vqa/SLAKE/InternVL3/eval_seed0/InternVL3-8B-hf,1,1,0,InternVL3-8B-hf,...,0.093509,0.601884,0.102192,0.128382,0.209306,0.711538,0.212497,0.238663,0.745192,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,VILA,vqa,VQA-RAD,general,ML,vqa/VQA-RAD/VILA1.5/eval_seed0/1epoch-lora8,1,1,1,train_lora_ML_seed42_vila,...,0.485218,0.470030,0.485218,0.472669,0.637450,0.637450,0.637450,0.637450,0.637450,44.820000
69,VILA-M3,vqa,VQA-RAD,medical,ML,vqa/VQA-RAD/VILA-M3/eval_seed0/1epoch-lora8,1,1,1,train_lora_ML_seed42_vila_m3,...,0.370001,0.420918,0.370242,0.379170,0.537849,0.537849,0.537849,0.537849,0.537849,46.795000
70,Lingshu,vqa,Harvard-FairVLMed10k,medical,ML,vqa/Harvard-FairVLMed10k/Lingshu/eval_seed0/1e...,1,1,1,train_lora_ML_seed42_lingshu,...,0.576751,0.591223,0.580907,0.580271,0.738978,0.738978,0.738978,0.738978,0.738978,57.234234
71,VILA,vqa,Harvard-FairVLMed10k,general,ML,vqa/Harvard-FairVLMed10k/VILA1.5/eval_seed0/1e...,1,1,1,train_lora_ML_seed42_vila,...,0.662567,0.638053,0.668046,0.647041,0.811122,0.811122,0.811122,0.811122,0.811122,56.039540


In [4]:
df_results["model"].value_counts()

model
Qwen25-VL    8
LLaVA-1.5    8
VILA-M3      8
VILA         8
Lingshu      8
LLaVA-Med    8
Qwen2-VL     7
Gemma3       7
InternVL3    7
MedGemma     4
Name: count, dtype: int64

In [208]:
def plot_raincloud(
    df_group1, df_group2, dataset=None, filename="raincloud", legend_label=None, target_metric="accuracy_closed"
):
    if dataset is not None:
        df_group1 = df_group1.loc[df_group1["dataset"] == dataset]
        df_group2 = df_group2.loc[df_group2["dataset"] == dataset]

    df_plot = pd.concat([df_group1, df_group2]).reset_index(drop=True)

    plt.figure(figsize=(8, 3))

    ax = sns.violinplot(
        x=target_metric,
        y="task",
        data=df_group1,
        inner=None,
        split=True,
        linewidth=1,
        color=(0.19460784, 0.45343137, 0.63284314),
    )
    for violin in ax.collections:
        facecolor = violin.get_facecolor()
        violin.set_facecolor((*facecolor[:3], 0.4))
    ax = sns.violinplot(
        x=target_metric,
        y="task",
        data=df_group2,
        inner=None,
        split=True,
        linewidth=1,
        color=[0.88186275, 0.50539216, 0.17303922],
    )
    for violin in ax.collections:
        facecolor = violin.get_facecolor()
        violin.set_facecolor((*facecolor[:3], 0.4))

    ax = sns.boxplot(
        x=target_metric,
        y="task",
        data=df_plot,
        # whis=1.5,
        width=0.15,
        gap=0.3,
        hue="model_type",
        hue_order=["general", "medical"],
    )

    if legend_label is not None:
        handles, _ = ax.get_legend_handles_labels()  # Get the artists.
        ax.legend(handles, legend_label, loc="best")

    plt.savefig(
        f"raincloud_plot/{filename}_{target_metric}.png", bbox_inches="tight")

    plt.close()


def stat_test(df_group1, df_group2, dataset=None, target_metric="accuracy_closed", alternative="greater"):
    if dataset is not None:
        df_group1 = df_group1.loc[df_group1["dataset"] == dataset]
        df_group2 = df_group2.loc[df_group2["dataset"] == dataset]

    U, p = mannwhitneyu(
        df_group1[target_metric].tolist(), df_group2[target_metric].tolist(), alternative=alternative, method="auto"
    )

    prob = U / (len(df_group1[target_metric].tolist())
                * len(df_group2[target_metric].tolist()))

    return U, p, prob

## RQ1

In [209]:
df_group1 = df_results.loc[(df_results["model_type"] == "general") & (
    df_results["trainable_module"] != "ML")]
df_group2 = df_results.loc[(df_results["model_type"] == "medical") & (
    df_results["trainable_module"] != "ML")]

In [210]:
for dataset in df_results["dataset"].unique():
    plot_raincloud(
        df_group1,
        df_group2,
        dataset=dataset,
        filename=f"RQ1_{dataset}",
        legend_label=["Off-the-shelf General", "Off-the-shelf Medical"],
    )

    U, p, prob = stat_test(df_group1, df_group2,
                           dataset=dataset, alternative="greater")
    print(f"RQ1, {dataset}: U={U}, p={p}, prob={prob}")

RQ1, SLAKE: U=8.0, p=0.8238095238095238, prob=0.3333333333333333
RQ1, PathVQA: U=5.0, p=0.9428571428571428, prob=0.20833333333333334
RQ1, VQA-RAD: U=8.0, p=0.8238095238095238, prob=0.3333333333333333
RQ1, Harvard-FairVLMed10k: U=4.0, p=0.9666666666666667, prob=0.16666666666666666


## RQ2

In [200]:
df_group1 = df_results.loc[(df_results["model_type"] == "general") & (
    df_results["trainable_module"] == "ML")]
df_group2 = df_results.loc[(df_results["model_type"] == "medical") & (
    df_results["trainable_module"] != "ML")]

In [201]:
for dataset in df_results["dataset"].unique():
    plot_raincloud(
        df_group1,
        df_group2,
        dataset=dataset,
        filename=f"RQ2_{dataset}",
        legend_label=["Tuned General", "Off-the-shelf Medical"],
    )

    U, p, prob = stat_test(df_group1, df_group2, dataset=dataset, alternative="greater")
    print(f"RQ2, {dataset}: U={U}, p={p}, prob={prob}")

RQ2, SLAKE: U=13.0, p=0.2777777777777778, prob=0.65
RQ2, PathVQA: U=22.5, p=0.016238201236439122, prob=0.9375
RQ2, VQA-RAD: U=5.0, p=0.9047619047619049, prob=0.25
RQ2, Harvard-FairVLMed10k: U=16.0, p=0.014285714285714285, prob=1.0


## RQ3

In [202]:
df_group1 = df_results.loc[(df_results["model_type"] == "general") & (
    df_results["trainable_module"] == "ML")]
df_group2 = df_results.loc[(df_results["model_type"] == "medical") & (
    df_results["trainable_module"] == "ML")]

In [204]:
for dataset in df_results["dataset"].unique():
    plot_raincloud(
        df_group1,
        df_group2,
        dataset=dataset,
        filename=f"RQ3_{dataset}",
        legend_label=["Tuned General", "Tuned Medical"],
    )

    U, p, prob = stat_test(df_group1, df_group2,
                           dataset=dataset, alternative="greater")
    print(f"RQ3, {dataset}: U={U}, p={p}, prob={prob}")

RQ3, SLAKE: U=13.0, p=0.2777777777777778, prob=0.65
RQ3, PathVQA: U=13.5, p=0.14983857960279828, prob=0.75
RQ3, VQA-RAD: U=8.0, p=0.5, prob=0.5333333333333333
RQ3, Harvard-FairVLMed10k: U=8.0, p=0.3142857142857143, prob=0.6666666666666666
